In [23]:
#pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [25]:
import pandas as pd
import numpy as np
import sqlite3 
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

## Neighbor 
#### Take-Home Exercise - Conversions
#### Yinan Kang 

#### Introduction
##### Data: 
* Reservations
* Search Events
* Listing Detail Views
* User IDs (Amplitude)
  
*Note*: I uploaded the '.csv' files into a '.db' file, then made adjustments such as joining 'user_id' to the Search and Listing tables, changing all date columns to datetime() format, filtering out bots, and other housekeeping to increase efficiency during the exercise. As a result, some FROM statements will feature slightly adjusted table names (e.g. 'good_search'). These are tables with the original data plus these housekeeping tweaks - the SQL queries to generate these tables will be in the Appendix. 

##### Tools Used: 
* SQLite: Used locally to explore and understand the data. Useful queries and manipulations will be included below.
    * SQLite uses slightly different syntax than more common SQL types (e.g. mySQL, Postgres), some of which are evident later in this notebook. 
* Python: Pandas, SKLearn, and Streamlit displayed via Jupyter (this notebook).

AI tools were used to assist in formatting queries post-development for compatibility with SQLite's syntax, as well as formatting model outputs in Python

#### Measure Baseline Conversion and Associated Trends

From an overall perspective, we can choose to measure conversion as: 

$$\text{Conversion Rate} = \frac{\text{Number of Conversions}}{\text{Number of Sessions}} \times 100$$

"Number of Conversions" can be calculated as the the number of distinct reservations for which payment was successfully processed. 

"Number of Sessions" can be calculated as the number of distinct sessions. This can be measured based on business requirements. Here, we can choose to define: *Session = A single or sequence of user activity with no more than 30 minutes of inactivity across searches, listing views, or bookings.*

The next cell contains the SQL queries to generate "Number of Conversions" and "Number of Sessions" as defined above, and the corresponding conversion rate. 

In [2]:
conn = sqlite3.connect('mydb.db')

In [11]:
# Read query results into a DataFrame
num_conversions = pd.read_sql_query(
"""
select count(distinct reservation_id)
from good_reservations
where successful_payment_collected_at is not NULL;
""", conn).iloc[0, 0]


num_sessions = pd.read_sql_query(
"""
with all_activities as (
  select user_id, event_time from good_search
  union 
  select user_id, event_time from good_listings
  union
  select renter_user_id as user_id, successful_payment_collected_at as event_time 
  from good_reservations
),
ordered_activities as (
  select 
    user_id,
    event_time,
    lag(event_time) over (partition by user_id order by event_time) as prev_event_time
  from all_activities
),
sessions_flagged as (
  select 
    user_id,
    event_time,
    prev_event_time,
    case 
      when prev_event_time is null then 1
      when (julianday(event_time) - julianday(prev_event_time)) * 24 * 60 > 30 then 1
      else 0
    end as is_new_session
  from ordered_activities
),
sessions_numbered as (
  select 
    user_id,
    event_time,
    sum(is_new_session) over (partition by user_id order by event_time) as session_id
  from sessions_flagged
)
select count(distinct user_id || '-' || session_id) as total_sessions
from sessions_numbered;
""", conn).iloc[0, 0]

print('Overall Conversion Rate by Sessions: ' + str(round(num_conversions / num_sessions * 100,2)) + '%')

Overall Conversion Rate by Sessions: 8.38%


**Alternatively**, we can look at conversion rate in terms of people who enter the funnel (search -> listing -> reservations) rather than sessions: 
$$\text{Conversion Rate} = \frac{\text{Number of People Who Book}}{\text{Total Number of People}} \times 100$$

In [13]:
conversion_rate_people = pd.read_sql_query(
"""
WITH total_bookers AS (
    SELECT COUNT(DISTINCT renter_user_id) as total_bookers
    FROM good_reservations
    WHERE payment_date IS NOT NULL
), 
total_people AS (
    SELECT COUNT(DISTINCT user_id) as total_people
    FROM good_search
)
SELECT 
    total_bookers * 100.0 / total_people as conversion_rate
FROM total_bookers, total_people;
""", conn).iloc[0,0]
print('Overall Conversion Rate by People: ' + str(round(conversion_rate_people,2)) + '%')

Overall Conversion Rate by People: 16.2%


#### Observe Action(s) Leading to Conversion

Let's dive deeper into the data to understand how customers are moving through the funnel. Any analytics here will be helpful in determining recommendations for improving conversion. 

First, it is observed that we have a strictly linear funnel [Searches -> Listings -> Reservations], meaning there are no instances of a user viewing listing without first making a search, and no reservations booked without first viewing a listing. With this in mind, we can evaluate the effectiveness of the various funnel stages and potentially identify opportunities. 

First, I will build out an aggregated summary table of user search and view counts and attributes, as well as converstion status. 

In [ ]:
'''
CREATE TABLE user_booking_funnel AS
SELECT 
    s.user_id,

    -- Booking outcome
    CASE WHEN r.renter_user_id IS NOT NULL THEN 1 ELSE 0 END as converted,
    MIN(r.payment_date) as first_booking_date,

    -- Activity counts
    COUNT(DISTINCT s.search_id) as total_searches,
    COUNT(DISTINCT l.listing_id) as total_listings_viewed,
	COUNT(DISTINCT r.reservation_id) as total_reservations,
    COUNT(DISTINCT s.search_dma) as unique_search_dmas,
    COUNT(DISTINCT l.click_dma) as unique_click_dmas,

    -- Timing metrics
    MIN(s.event_time) as first_search_date,
    MAX(s.event_time) as last_search_date,
    MIN(l.event_time) as first_listing_view_date,
    MAX(l.event_time) as last_listing_view_date,
    JULIANDAY(MIN(r.payment_date)) - JULIANDAY(MIN(s.event_time)) as days_to_conversion,

    -- First touch attribution
    (SELECT s2.first_attribution_source 
     FROM good_search s2 
     WHERE s2.user_id = s.user_id 
     ORDER BY s2.event_time 
     LIMIT 1) as first_attribution_source,
    (SELECT s2.first_attribution_channel 
     FROM good_search s2 
     WHERE s2.user_id = s.user_id 
     ORDER BY s2.event_time 
     LIMIT 1) as first_attribution_channel
FROM good_search s
LEFT JOIN good_listings l ON s.user_id = l.user_id AND s.search_id = l.search_id
LEFT JOIN good_reservations r ON s.user_id = r.renter_user_id AND r.payment_date IS NOT NULL
GROUP BY s.user_id;
'''

In [3]:
funnel_df_no_outliers = pd.read_sql_query("select * from user_booking_funnel", conn)
funnel_df_no_outliers.head(5)

,user_id,converted,first_booking_date,total_searches,total_listings_viewed,total_reservations,unique_search_dmas,unique_click_dmas,first_search_date,last_search_date,first_listing_view_date,last_listing_view_date,days_to_conversion,first_attribution_source,first_attribution_channel
0,1,0,None,15,0,0,7,0,2023-02-25 03:05:51,2023-11-28 00:08:54,None,None,NaN,direct,direct
1,2,0,None,8,0,0,6,0,2023-02-03 21:54:42,2023-11-26 20:09:02,None,None,NaN,direct,direct
2,3,1,2023-02-18 10:50:44,9,14,6,6,6,2023-01-27 10:36:40,2023-10-06 23:51:55,2023-01-27 10:38:43,2023-10-06 23:56:10,22.009769,organic,google maps
3,4,0,None,15,0,0,7,0,2023-02-02 17:45:54,2023-12-18 16:27:09,None,None,NaN,paid,google ads
4,5,0,None,8,0,0,6,0,2023-02-05 23:30:10,2023-11-26 12:03:33,None,None,NaN,paid,google ads


Let's create and contrast two profiles of visitors, divided simply by whether their 'converted' status is 0 or 1. 

In [5]:
converted_df = funnel_df[funnel_df['converted'] == 1] 
unconverted_df = funnel_df[funnel_df['converted'] == 0] 

In [6]:
len(converted_df)/len(funnel_df) # check if we see same people conversion rate as earlier, which we do 

0.162015503875969

We can first focus on analyzing differences in the customer journey between customers who have converted and for those who have not. There are different ways to slice the data for this topic, but I will conduct the one described below. 

* Filter away outliers in 'total_search' to make subsequent visualizations easier to interpret
* Comparison 1 - compare 'total_search' between converted and unconverted customers. Do we see any indication unconverted customers are dropping out or getting stuck in this part of the funnel? 
* Comparison 2 - converted customers might make more than one reservation, meaning multiple separate booking endeavors. While unconverted customers may also have multiple booking endeavors, that is less certain. We can compare 'total_searches / total_reservations' for converted customers, against 'total_searches' for unconverted customers. 

There are a couple of far-out outliers within the 'total_searches' column that we can filter out to make general analysis more succinct. This will be followed by Comparison 1. 

In [19]:
Q1 = funnel_df['total_searches'].quantile(0.25)
Q3 = funnel_df['total_searches'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

funnel_df_no_outliers = funnel_df[(funnel_df['total_searches'] >= lower_bound) & 
                                   (funnel_df['total_searches'] <= upper_bound)]

print(f"Original size: {len(funnel_df)}")
print(f"After removing outliers: {len(funnel_df_no_outliers)}")

Original size: 9030
After removing outliers: 8877


In [20]:
fig = px.box(
    funnel_df_no_outliers, 
    x='user_type', 
    y='total_searches',
    color='user_type',
    title='Total Searches: Converted vs Unconverted',
    labels={'total_searches': 'Total Searches', 'user_type': 'User Type'}
)
fig.show()

In [21]:
funnel_df['user_type'] = funnel_df['converted'].map({0: 'Unconverted', 1: 'Converted'})
fig = px.histogram(
    funnel_df_no_outliers, 
    x='total_searches', 
    color='user_type',
    barmode='overlay',
    title='Distribution of Total Searches',
    labels={'total_searches': 'Total Searches', 'user_type': 'User Type'},
    opacity=0.7
)
fig.show()

From the above point-of-view, both converted and unconverted customers seem to generally have a similar search experience - it is not the case that unconverted customers get stuck or leave the search based on this viewpoint. 

Let's take a look at Comparison 2. 

In [ ]:
# Create the new column for converted users
funnel_df_no_outliers['searches_per_reservation'] = None  # Initialize
funnel_df_no_outliers.loc[funnel_df_no_outliers['converted'] == 1, 'searches_per_reservation'] = (
    funnel_df_no_outliers.loc[funnel_df_no_outliers['converted'] == 1, 'total_searches'] / 
    funnel_df_no_outliers.loc[funnel_df_no_outliers['converted'] == 1, 'total_reservations']
)

# Create separate dataframes for plotting
converted_df = funnel_df_no_outliers[funnel_df_no_outliers['converted'] == 1][['searches_per_reservation']].copy()
converted_df['metric'] = 'Searches per Reservation (Converted)'
converted_df['value'] = converted_df['searches_per_reservation']

unconverted_df = funnel_df_no_outliers[funnel_df_no_outliers['converted'] == 0][['total_searches']].copy()
unconverted_df['metric'] = 'Total Searches (Unconverted)'
unconverted_df['value'] = unconverted_df['total_searches']

# Combine for plotting
plot_df = pd.concat([
    converted_df[['value', 'metric']], 
    unconverted_df[['value', 'metric']]
])

In [24]:
fig = px.box(
    plot_df,
    x='metric',
    y='value',
    color='metric',
    title='Searches per Reservation (Converted) vs Total Searches (Unconverted)',
    labels={'value': 'Value', 'metric': 'Metric'}
)
fig.show()

Interesting - the behavior of a converted customer on search is quite a bit different once you take booking actions into account. Converted customers appear to be ready to book after just a few searches, thus indicating maybe completing the funnel shortens future conversions.

Another item we can look into w.r.t. the search piece of the funnel are aggregate statistics on what search attributes and parameters lead to the most listings viewed and subsequent conversions. The following table can be created and would be accessible for both analyst and business teams alike.  

In [ ]:
'''
CREATE TABLE search_attribute_conversion AS
SELECT 
    s.search_type,
    s.search_term,
    s.search_dma,
    s.first_attribution_source,
    s.first_attribution_channel,
    
    -- Search-level metrics
    COUNT(DISTINCT s.search_id) as total_searches,
    COUNT(DISTINCT s.user_id) as unique_users,
    COUNT(DISTINCT l.listing_id) as total_listings_viewed,
    COUNT(DISTINCT l.listing_id) * 1.0 / COUNT(DISTINCT s.search_id) as avg_listings_per_search,
    
    -- Conversion metrics
    COUNT(DISTINCT CASE WHEN r.payment_date IS NOT NULL THEN r.renter_user_id END) as converted_users,
    COUNT(DISTINCT CASE WHEN r.payment_date IS NOT NULL THEN r.renter_user_id END) * 100.0 / 
        COUNT(DISTINCT s.user_id) as conversion_rate,
    
    -- Geography conversion
    COUNT(DISTINCT CASE WHEN r.payment_date IS NOT NULL THEN r.dma END) as booking_dmas

FROM good_search s
LEFT JOIN good_listings l ON s.search_id = l.search_id
LEFT JOIN good_reservations r ON s.user_id = r.renter_user_id AND r.payment_date IS NOT NULL
GROUP BY s.search_type, s.search_term, s.search_dma, 
         s.first_attribution_source, s.first_attribution_channel;
'''

In [ ]:
search_attribute_conversion_df = pd.read_sql_query("select * from search_attribute_conversion", conn)
search_attribute_conversion_df.head(10)

We can also take a short dive into the [listing -> reservation] piece of the funnel. A good start would be to observe any relationship between a listing's search_position and the likelihood of its viewing being a conversion. 

In [3]:
position_conversion_df = pd.read_sql_query(
"""
    SELECT 
    l.search_position,
    COUNT(DISTINCT l.user_id || '-' || l.listing_id) as total_views,
    COUNT(DISTINCT r.reservation_id) as conversions,
    ROUND(100.0 * COUNT(DISTINCT r.reservation_id) / COUNT(DISTINCT l.user_id || '-' || l.listing_id), 2) as conversion_rate_pct
FROM good_listings l
LEFT JOIN good_reservations r 
    ON l.user_id = r.renter_user_id 
    AND l.listing_id = r.listing_id
    AND r.payment_date IS NOT NULL
GROUP BY l.search_position
ORDER BY l.search_position
""", conn) 
position_conversion_df

,search_position,total_views,conversions,conversion_rate_pct
0,1,6391,648,10.14
1,10,5257,460,8.75
2,11,5224,475,9.09
3,12,5217,521,9.99
4,13,5232,473,9.04
5,14,4954,458,9.25
6,15,5179,525,10.14
7,16,5112,514,10.05
8,17,5075,499,9.83
9,18,4892,451,9.22


The conversion data is remarkably similar across the various search positions, a trend which that may or may not be a good indicator for us. 

#### Predictive Model for Conversion

One thing that is always interesting to do, though its impact would need business and product support, is put the opening pieces together for a predictive model. 

We can build a simple one at the session-level, as we have the most data for that granularity. 

First, I a new features table needs to be created. There will be some intermediate tables created as well, joining and aggregating the various available metrics of the sales funnel. 

In [9]:
conn.execute(
"""
CREATE TABLE session_funnel_metrics AS
WITH all_events AS (
    -- All searches
    SELECT 
        user_id,
        event_time,
        search_id,
        search_term,
        NULL as listing_id,
        NULL as reservation_date,
        'search' as event_type
    FROM good_search
    
    UNION ALL
    
    -- All listing views
    SELECT 
        user_id,
        event_time,
        search_id,
        NULL as search_term,
        listing_id,
        NULL as reservation_date,
        'listing_view' as event_type
    FROM good_listings
    
    UNION ALL
    
    -- All reservations
    SELECT 
        renter_user_id as user_id,
        payment_date as event_time,
        NULL as search_id,
        NULL as search_term,
        NULL as listing_id,
        payment_date as reservation_date,
        'reservation' as event_type
    FROM good_reservations
    WHERE payment_date IS NOT NULL
),
events_with_prev AS (
    SELECT 
        user_id,
        event_time,
        event_type,
        search_id,
        search_term,
        listing_id,
        reservation_date,
        LAG(event_time) OVER (PARTITION BY user_id ORDER BY event_time) as prev_event_time
    FROM all_events
),
sessions AS (
    SELECT 
        user_id,
        event_time,
        event_type,
        search_id,
        search_term,
        listing_id,
        reservation_date,
        SUM(CASE 
            WHEN prev_event_time IS NULL THEN 1
            WHEN (JULIANDAY(event_time) - JULIANDAY(prev_event_time)) > (30.0/1440) THEN 1
            ELSE 0
        END) OVER (PARTITION BY user_id ORDER BY event_time ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as session_number
    FROM events_with_prev
),
session_ids AS (
    SELECT 
        user_id,
        session_number,
        user_id || '_' || CAST(session_number AS TEXT) as session_id,
        MIN(event_time) as session_start,
        MAX(event_time) as session_end
    FROM sessions
    GROUP BY user_id, session_number
)
SELECT 
    si.session_id,
    si.user_id,
    si.session_start,
    si.session_end,
    ROUND((JULIANDAY(si.session_end) - JULIANDAY(si.session_start)) * 1440, 2) as session_duration_minutes,
    
    -- Activity counts
    COUNT(DISTINCT s.search_id) as num_searches,
    COUNT(DISTINCT CASE WHEN s.event_type = 'search' THEN s.event_time END) as num_visits,
    COUNT(DISTINCT s.search_term) as num_search_terms,
    COUNT(DISTINCT s.listing_id) as num_listings_viewed,
    COUNT(DISTINCT s.reservation_date) as num_bookings

FROM session_ids si
LEFT JOIN sessions s ON si.user_id = s.user_id AND si.session_number = s.session_number
GROUP BY si.session_id, si.user_id, si.session_start, si.session_end
ORDER BY si.session_start;
""")

OperationalError: table session_funnel_metrics already exists

In [41]:
conn.execute("""create table session_conversion_features as SELECT 
    sfm.session_id,
    sfm.user_id,
    sfm.num_bookings > 0 as target,
    
    -- Session activity (already calculated in session_funnel_metrics)
    sfm.num_searches,
    sfm.num_search_terms,
    sfm.num_listings_viewed,
    sfm.session_duration_minutes,
    CAST(sfm.num_listings_viewed AS FLOAT) / NULLIF(sfm.num_searches, 0) as listings_per_search_session,
    
    -- Temporal features
    CAST(strftime('%H', sfm.session_start) AS INTEGER) as session_hour,
    CAST(strftime('%w', sfm.session_start) AS INTEGER) as session_day_of_week,
    
    -- User history features (prior to this session)
    COUNT(DISTINCT prior.session_id) as prior_session_count,
    COALESCE(SUM(prior.num_searches), 0) as prior_total_searches,
    COALESCE(SUM(prior.num_listings_viewed), 0) as prior_total_listings,
    COALESCE(SUM(prior.num_bookings), 0) as prior_bookings,
    
    -- Engagement intensity flags
    CASE WHEN sfm.num_listings_viewed >= 3 THEN 1 ELSE 0 END as viewed_multiple_listings,
    CASE WHEN sfm.num_search_terms > 1 THEN 1 ELSE 0 END as diverse_search_terms,
    CASE WHEN sfm.session_duration_minutes > 10 THEN 1 ELSE 0 END as long_session

FROM session_funnel_metrics sfm
LEFT JOIN session_funnel_metrics prior 
    ON sfm.user_id = prior.user_id 
    AND prior.session_start < sfm.session_start
GROUP BY sfm.session_id;""")

We can now create a 'model_df' which will include the 'target' [0 or 1 if conversion], as well as other calculated characteristics of all the sessions in our dataset that we tied together. 

In [17]:
model_df = pd.read_sql_query("select * from session_conversion_features", conn)
model_df.head(10)

,session_id,user_id,target,num_searches,num_search_terms,num_listings_viewed,session_duration_minutes,listings_per_search_session,session_hour,session_day_of_week,prior_session_count,prior_total_searches,prior_total_listings,prior_bookings,viewed_multiple_listings,diverse_search_terms,long_session
0,1000_1,1000,0,1,1,0,0.0,0.0,13,4,0,0,0,0,0,0,0
1,1000_10,1000,0,1,1,0,0.0,0.0,2,5,9,9,0,0,0,0,0
2,1000_11,1000,0,1,1,0,0.0,0.0,21,0,10,10,0,0,0,0,0
3,1000_12,1000,0,1,1,0,0.0,0.0,14,0,11,11,0,0,0,0,0
4,1000_13,1000,0,1,1,0,0.0,0.0,14,2,12,12,0,0,0,0,0
5,1000_14,1000,0,1,1,0,0.0,0.0,15,6,13,13,0,0,0,0,0
6,1000_2,1000,0,1,1,0,0.0,0.0,21,1,1,1,0,0,0,0,0
7,1000_3,1000,0,1,1,0,0.0,0.0,8,2,2,2,0,0,0,0,0
8,1000_4,1000,0,1,1,0,0.0,0.0,8,6,3,3,0,0,0,0,0
9,1000_5,1000,0,1,1,0,0.0,0.0,13,4,4,4,0,0,0,0,0


The following cells create train-test splits and model training. 

In [26]:
# Drop ID columns and target
feature_cols = [
    'num_searches',
    'num_search_terms', 
    'num_listings_viewed',
    'session_duration_minutes',
    'listings_per_search_session',
    'session_hour',
    'session_day_of_week',
    'prior_session_count',
    'prior_total_searches',
    'prior_total_listings',
    'prior_bookings',
    'viewed_multiple_listings',
    'diverse_search_terms',
    'long_session'
]

X = model_df[feature_cols].copy()
y = model_df['target'].copy()

# Handle any missing or infinite values
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)

print(f"\nFeatures shape: {X.shape}")
print(f"Target distribution:\n{y.value_counts()}")


Features shape: (123662, 14)
Target distribution:
target
0    113799
1      9863
Name: count, dtype: int64


In [28]:
# Split data (stratify to maintain conversion rate in both sets)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"\nTrain set: {len(X_train)} sessions")
print(f"Test set: {len(X_test)} sessions")


Train set: 98929 sessions
Test set: 24733 sessions


In [29]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [30]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train logistic regression
model = LogisticRegression(
    max_iter=1000,
    random_state=42,
    class_weight='balanced',  # Important for imbalanced data
    C=1.0  # Regularization strength (lower = more regularization)
)

model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

In [31]:
# Evaluate performance
print("\n" + "="*50)
print("MODEL PERFORMANCE")
print("="*50)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['No Conversion', 'Conversion']))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(f"\nTrue Negatives: {cm[0,0]}")
print(f"False Positives: {cm[0,1]}")
print(f"False Negatives: {cm[1,0]}")
print(f"True Positives: {cm[1,1]}")

auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"\nROC-AUC Score: {auc_score:.4f}")


MODEL PERFORMANCE

Classification Report:
               precision    recall  f1-score   support

No Conversion       1.00      1.00      1.00     22760
   Conversion       1.00      1.00      1.00      1973

     accuracy                           1.00     24733
    macro avg       1.00      1.00      1.00     24733
 weighted avg       1.00      1.00      1.00     24733


Confusion Matrix:
[[22760     0]
 [    0  1973]]

True Negatives: 22760
False Positives: 0
False Negatives: 0
True Positives: 1973

ROC-AUC Score: 1.0000


Our model is perfect - normally, this would be a huge concern. However, the datasets we are using all generally have signs of being synthetic data (nearly ideal distributions across every relevant field). Most likely, that is the primary contributor to why a model is able to delineate so cleanly. 

Just out of curiosity, we can check for obvious symptoms of data leakage or intracorrelation. 

In [44]:
correlations = X_train.corrwith(pd.Series(y_train))
print("Feature correlations with target:")
print(correlations.sort_values(ascending=False))

# Check if any single feature perfectly predicts target
for col in X_train.columns:
    perfect = (X_train[col] == y_train).mean()
    if perfect > 0.95:
        print(f"\n⚠️ {col} perfectly predicts target {perfect:.1%} of the time")


Feature correlations with target:
prior_bookings                 0.411927
session_day_of_week            0.004399
session_hour                   0.001318
long_session                  -0.040267
diverse_search_terms          -0.045084
prior_total_listings          -0.053842
prior_session_count           -0.054374
prior_total_searches          -0.055442
session_duration_minutes      -0.103576
viewed_multiple_listings      -0.107634
num_listings_viewed           -0.208251
listings_per_search_session   -0.213046
num_searches                  -0.611191
num_search_terms              -0.692607
dtype: float64


In [45]:
# Feature importance
print("\n" + "="*50)
print("FEATURE IMPORTANCE (Coefficients)")
print("="*50)

feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'coefficient': model.coef_[0],
    'abs_coefficient': np.abs(model.coef_[0]),
    'odds_ratio': np.exp(model.coef_[0])  # Exponentiate to get odds ratios
}).sort_values('abs_coefficient', ascending=False)

print("\nAll Features Ranked by Importance:")
print(feature_importance.to_string(index=False))

print("\n" + "="*50)
print("INTERPRETATION:")
print("="*50)
print("- Positive coefficient = increases conversion probability")
print("- Negative coefficient = decreases conversion probability")
print("- Odds ratio > 1 = increases odds of conversion")
print("- Odds ratio < 1 = decreases odds of conversion")


FEATURE IMPORTANCE (Coefficients)

All Features Ranked by Importance:
                    feature  coefficient  abs_coefficient  odds_ratio
           num_search_terms    -3.993146         3.993146    0.018442
               num_searches    -3.516227         3.516227    0.029711
               long_session     1.421861         1.421861    4.144827
       diverse_search_terms     0.840758         0.840758    2.318124
   session_duration_minutes     0.678501         0.678501    1.970920
             prior_bookings     0.566899         0.566899    1.762792
listings_per_search_session    -0.365516         0.365516    0.693838
               session_hour     0.252970         0.252970    1.287845
        num_listings_viewed    -0.164266         0.164266    0.848516
        prior_session_count    -0.085074         0.085074    0.918444
       prior_total_searches    -0.083091         0.083091    0.920268
   viewed_multiple_listings    -0.078855         0.078855    0.924174
       prior_total_

There does not appear to be a smoking gun feature that is providing unquestonable data leakage. It is likely to be a combination of the features and their (likely) synthetic nature. There is also the possibility or errors in the SQL queries building the feature table, of course, which would certainly be an area of focus upon further review. 

We do learn that higher 'num_search_terms' and 'num_searches' are by far the most impactful features that we've created, which will influence our final recommendations below. 

#### Summary and Recommendations

Throughout the exercise, the following have been covered: 
* Defining Overall Conversion Rate, and Calculating
* Recognizing and Studying Actions in the Funnel, and how they relate to Conversion
* Building Predictive Model for Conversion at the Session-level

Here would be the primary recommendations based on findings: 
1. Improve search relevance. Early in the notebook, we observed that converted customers move quickly through future search sessions during future bookings, while unconverted customers remain on higher volumes of searches without conversion. During the modeling portion of the notebook, we also see what 'num_search_terms' and 'num_searches' are the primary negative drivers in our conversion model. Next step would be to conduct further analyses and experimentation to achieve this.
2. Double-down on building an accurate conversion model, either at the session-level, user-level, or both. There are many elements of the data that were not included in the preliminary model built above (e.g. locations, IP addresses, etc.) that may be beneficial. A robust conversion model can serve as the basis for product enhancements and future experimentation.

Additional data that would help: budget, cost, and metrics for listing quality. This would form a basis of prioritization (budget, cost) which could can be used to established weighted features. Listing quality, and similar metrics, would further distinguish user experiences across the funnel data.

#### Appendix

SQL Queries of the enhanced tables created (good_search, good_listings, good_reservations) were created in roughly the same way. Here is an example for the table good_search: 

In [48]:
%%capture
"""
create table good_search as 
select u.user_id, datetime(event_time), s.*
from search s
left join user u 
on s.merged_amplitude_id = user.merged_amplitude_id 
where is_box = 'False'
"""